First, I am modelling a Perceptron, which forms a simple neural network with one neuron.
With this Perceptron, I am trying to model the basic logic gates `AND`, `OR`, and `NOT`.

In [27]:
AND = [(0, 0, 0), (0, 1, 0), (1, 0, 0), (1, 1, 1)]
OR = [(0, 0, 0), (0, 1, 1), (1, 0, 1), (1, 1, 1)]
NOT = [(0, 1), (1, 0)]

In [28]:
"""
Some helper functions, including activation functions which can be used,
namely the heaviside function and basic sign function.
"""
def heaviside(z: int):
    return 1 if z >= 0 else 0

def sign(z: int):
    return 1 if z > 0 else -1

def relu(z: int):
    return max(0, z)

def hw(w: list, point: tuple, activation):
    return activation(sum([x*y for x,y in zip(w,point[:len(point)-1])]))

In [29]:
"""
The Perceptron Learning Algorithm, default learning rate = 0.1
Returns weights
"""
def perceptron_learning_algorithmn(data: list[tuple], rate=0.1, activation=heaviside):
    n = len(data[0])
    data = [(1,) + tup for tup in data] # initialize dummy variable = 1 for each point
    w = [0.0 for i in range(n)] # initialize weights = 0
    converged = False

    for i in range(100):
      converged = True
      for point in data:
          # predict yhat
          yhat = hw(w, point, activation)
          y = point[-1]

          # update weights if misclassified
          if yhat != y:
              converged = False
              step = (y - yhat) * rate
              for i in range(n):
                  w[i] = w[i] + step * point[i]
      if converged:
          break
    
    return w

"""
The Perceptron which uses the weights from the Perceptron Learning Algorithm
Returns the predicted values of the target variable
"""
def perceptron(data: list[tuple], rate=0.1, activation=heaviside):
    w = perceptron_learning_algorithmn(data, rate, activation)
    data = [(1,) + tup for tup in data] # add dummy variable = 1 to each point
    output = []
    for point in data:
        output.append(hw(w, point, activation))
    return output


In [30]:
"""
Helper function to test the results
"""
def test_perceptron(data: list[tuple], predicted: list):
    for i in range(len(data)):
        yhat = predicted[i]
        y = data[i][-1]
        correct = y == yhat

        print(f"POINT {data[i]}: PREDICTED = {yhat}, ACTUAL = {y}, RESULT = {y==yhat}")
        if y != yhat:
            correct = False
    print("TRUE" if correct else "FALSE")

And these are the weights returned by my Perceptron Learning Algorithm:
`AND` gate: [-0.2, 0.2, 0.1]
`OR` gate: [-0.1, 0.1, 0.1]
`NOT` gate: [0.0, -0.1]
The first weight in each array is the bias term.

In [31]:
predicted = perceptron(AND)
test_perceptron(AND, predicted)

predicted = perceptron(OR)
test_perceptron(OR, predicted)

predicted = perceptron(NOT)
test_perceptron(NOT, predicted)

POINT (0, 0, 0): PREDICTED = 0, ACTUAL = 0, RESULT = True
POINT (0, 1, 0): PREDICTED = 0, ACTUAL = 0, RESULT = True
POINT (1, 0, 0): PREDICTED = 0, ACTUAL = 0, RESULT = True
POINT (1, 1, 1): PREDICTED = 1, ACTUAL = 1, RESULT = True
TRUE
POINT (0, 0, 0): PREDICTED = 0, ACTUAL = 0, RESULT = True
POINT (0, 1, 1): PREDICTED = 1, ACTUAL = 1, RESULT = True
POINT (1, 0, 1): PREDICTED = 1, ACTUAL = 1, RESULT = True
POINT (1, 1, 1): PREDICTED = 1, ACTUAL = 1, RESULT = True
TRUE
POINT (0, 1): PREDICTED = 1, ACTUAL = 1, RESULT = True
POINT (1, 0): PREDICTED = 0, ACTUAL = 0, RESULT = True
TRUE


Now, I will try creating a Multi-Layered Neural Network by combining multiple Perceptrons. Then, I will implement the `XOR` gate, which can't be implemented with only 1 Perceptron as the data is not linearly separable.

![gatexor](gatexor.png)

In [32]:
XOR = [(0, 0, 0), (0, 1, 1), (1, 0, 1), (1, 1, 0)]

weights_and = perceptron_learning_algorithmn(AND)
weights_or = perceptron_learning_algorithmn(OR)
weights_not = perceptron_learning_algorithmn(NOT)

In [33]:
# Step 1: Generate Intermediate Data
xor_training_data = []
for a, b, target in XOR:
    or_out = hw(weights_or, (1, a, b), relu)
    and_out = hw(weights_and, (1, a, b), relu)
    not_out = hw(weights_not, (1, and_out), relu)
    xor_training_data.append((or_out, not_out, target))

# Step 2: Train Final Layer
weights_xor = perceptron_learning_algorithmn(xor_training_data)

# Step 3: Prediction Function
def predict_xor(a: int, b: int, activation=heaviside) -> int:
    or_out = hw(weights_or, (1, a, b), activation)
    and_out = hw(weights_and, (1, a, b), activation)
    not_out = hw(weights_not, (1, and_out), activation)
    return hw(weights_xor, (1, or_out, not_out), activation)

# Step 4: Test
print("Testing XOR Gate:")
for a, b, _ in XOR:
    print(f"({a}, {b}) → {predict_xor(a, b)}")

Testing XOR Gate:
(0, 0) → 0
(0, 1) → 0
(1, 0) → 0
(1, 1) → 0
